# [Create Agent with Azure Functions](https://learn.microsoft.com/en-us/python/api/overview/azure/ai-projects-readme?view=azure-python-preview#create-agent-with-azure-function-call)
AzureFunctionTool contains the input and output queues of azure function and the description of input parameters.

Inspired by [sample_agents_azure_functions.py](https://github.com/Azure/azure-sdk-for-python/blob/azure-ai-projects_1.0.0b4/sdk/ai/azure-ai-projects/samples/agents/sample_agents_azure_functions.py)

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-0513" # https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=azurecli%2Cpython&pivots=overview-azure-ai-search
index_name =  "ms-surface-specs"

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

Project Connection String: <...mai04-rg;mmai-hub04-prj01-fvye>


In [2]:
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import AzureFunctionTool, AzureFunctionStorageQueue # <<<<<<<<<<<<<<< SPECIFIC FOR AZURE FUNCTIONS
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

{'subscription_id': 'eca2eddb-0f0c-4351-a634-52751499eeea',
 'resource_group_name': 'mmai04-rg',
 'project_name': 'mmai-hub04-prj01-fvye'}

In [ ]:
# define some custom functions

from datetime import date

def save_file(joke:str) -> str:
    """
    Saves a text file using the textual content passed in the joke variable.
    Args:
        joke (str): The text content to be saved into the file. 
        
    Returns:
        Optional[str]: Returns 'success: file saved' on success, otherwise returns an error message.
    """
    file_name = "joke.txt"
    try:
        with open(file_name, "w") as f:
            f.write(joke)
        return "success: file saved"
    except Exception as e:
        return f"error: {str(e)}"


def get_flights(date_1:date, date_2:date) -> int:
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)


def my_cat_born_date() -> date:
    """ Returns my cat's born date """
    import datetime, random, json
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)

In [ ]:
from typing import Any, Callable, Set

user_functions: Set[Callable[..., Any]] = {
    save_file, 
    get_flights, 
    my_cat_born_date
}

user_functions

In [ ]:
# Let's use the "user_function" set, to call its functions

for function in user_functions:    
    if function.__name__ == "my_cat_born_date":
        cat_birthday = function() 
        print(f"My cat's birth date is {cat_birthday}")
    elif function.__name__ == "save_file": 
        result = function("This is the content")
        print(f"save_file() returned <{result}>")
    elif function.__name__ == "get_flights": 
        result = function("2015-01-01", "2021-04-04")
        print(f"get_flights() returned <{result}>")

In [ ]:
functions = FunctionTool(user_functions)
toolset = ToolSet()
toolset.add(functions)
print(f"toolset.definitions: {toolset.definitions}")
print(f"toolset.resources: {toolset.resources}")

In [ ]:
# Check if we already have any agents defined
project_client.agents.list_agents()

In [ ]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="You are a helpful assistant",
    toolset=toolset
)

agent.items

In [ ]:
# Create agent with AI search tool and process assistant run
agent = project_client.agents.create_agent(
    model=model_name,
    name="my-assistant",
    instructions="You are a helpful assistant",
    toolset=toolset
)

agent.items

In [ ]:
# Check again if the agent created above is now present here
project_client.agents.list_agents()

In [ ]:
# Create thread for communication
thread = project_client.agents.create_thread()
print(f"Created thread: {thread}")

In [ ]:
# Create message to thread
message = project_client.agents.create_message(
    thread_id=thread.id,
    role="user",
    content="Please write into a file the nr of flights between my cat born date and Easter 2021.",
)
print(f"Created message: {message}")

In [ ]:
%%time
# Create and process agent run in thread with tools
run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
print(f"Run finished with status: {run.status}. Run: {run}")

if run.status == "failed":
    # Check if you got "Rate limit is exceeded.", then you want to get more quota
    print(f"Run failed: {run.last_error}")

In [ ]:
# Fetch and log all messages
messages = project_client.agents.list_messages(thread_id=thread.id)
for m in messages.text_messages:
    print(m.text)

In [ ]:
# Get the last message from the sender
last_msg = messages.get_last_text_message_by_sender("assistant")
if last_msg:
    print(f"Last Message: {last_msg.text.value}")

In [ ]:
run_steps = project_client.agents.list_run_steps(run_id=run.id, thread_id=thread.id)
run_steps_data = run_steps['data']
for rs in run_steps["data"]:
    print(rs, '\n')

# START teardown

In [ ]:
project_client.agents.list_agents()

In [ ]:
print(f"deleting trhead: {thread}...")
project_client.agents.delete_thread(thread.id)

In [ ]:
# Delete all agents
for pca in project_client.agents.list_agents()['data']:
    print(f"Deleting agent {pca.name} ({pca.id})...")
    project_client.agents.delete_agent(pca.id)

# HIC SUNT LEONES